In [32]:
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from preprocessing import plot_cm

In [3]:
news_0 = pd.read_csv(r'C:\Users\darias\Desktop\news_0.csv',  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_0['class'] = 0

news_1 = pd.read_csv(r'C:\Users\darias\Desktop\news_1.csv',  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_1['class'] = 1

news_2 = pd.read_csv(r'C:\Users\darias\Desktop\news_2.csv',  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_2['class'] = 2

news_3 = pd.read_csv(r'C:\Users\darias\Desktop\news_3.csv',  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_3['class'] = 3

In [4]:
frames = [news_0, news_1, news_2, news_3]
data = pd.concat(frames).drop(["CODE"], axis=1)
data = data.sample(frac = 1)
display(data)

,L_TYPE_TOKEN_RATIO_LEMMAS,L_PROPER_NAME,L_PERSONAL_NAME,L_PUNCT,L_PUNCT_COM,L_PUNCT_SEMC,L_PUNCT_COL,L_PUNCT_DASH,L_CONT_A,L_FUNC_A,...,SY_NON_FINITE,SY_QUOTATIONS,SY_EXCLAMATION,SY_QUESTION,SY_ELLIPSES,SY_POSITIONING,SY_CONDITIONAL,SY_IMPERATIVE,SY_AMPLIFIED_SENT,class
59,0.512415,0.031603,0.009029,0.142212,0.072235,0.0,0.000000,0.004515,0.632054,0.209932,...,0.004515,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1
4070,0.527638,0.031826,0.023451,0.144054,0.051926,0.0,0.001675,0.010050,0.603015,0.207705,...,0.005025,0.000000,0.0,0.000000,0.001675,0.000000,0.000000,0.000000,0.0,1
964,0.493783,0.087034,0.055062,0.206039,0.087034,0.0,0.001776,0.000000,0.564831,0.223801,...,0.014210,0.039076,0.0,0.003552,0.001776,0.000000,0.003552,0.000000,0.0,0
5385,0.649573,0.017094,0.000000,0.085470,0.025641,0.0,0.000000,0.000000,0.709402,0.196581,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3
3113,0.600000,0.086957,0.000000,0.165217,0.052174,0.0,0.017391,0.000000,0.643478,0.191304,...,0.008696,0.034783,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,0.593939,0.103030,0.066667,0.187879,0.060606,0.0,0.018182,0.000000,0.654545,0.157576,...,0.012121,0.030303,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2
87,0.534527,0.112532,0.058824,0.176471,0.069054,0.0,0.005115,0.000000,0.626598,0.189258,...,0.005115,0.020460,0.0,0.000000,0.000000,0.005115,0.000000,0.002558,0.0,2
2498,0.538043,0.070652,0.076087,0.168478,0.043478,0.0,0.005435,0.005435,0.510870,0.184783,...,0.016304,0.000000,0.0,0.000000,0.010870,0.000000,0.000000,0.000000,0.0,1
6615,0.617143,0.114286,0.051429,0.182857,0.022857,0.0,0.005714,0.000000,0.662857,0.148571,...,0.005714,0.057143,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3


In [14]:
X = data.drop(["class"], axis=1)
y = data['class'].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.1, random_state=66)

In [16]:
# Normalize the data
sc = StandardScaler()
normed_train_data = pd.DataFrame(sc.fit_transform(X_train), columns = X.columns)
normed_test_data = pd.DataFrame(sc.fit_transform(X_test), columns = X.columns)
normed_val_data = pd.DataFrame(sc.fit_transform(X_val), columns = X.columns)

In [17]:
# Encode the categories
encoder = OneHotEncoder()
encoder.fit(y_train.reshape(-1, 1))
train_l = encoder.transform(y_train.reshape(-1, 1)).toarray()

In [19]:
encoder.fit(y_val.reshape(-1, 1))
val_l = encoder.transform(y_val.reshape(-1, 1)).toarray()

In [18]:
encoder.fit(y_test.reshape(-1, 1))
test_l = encoder.transform(y_test.reshape(-1, 1)).toarray()

In [46]:
def build_model():
    x = Input(shape=(101), dtype=tf.float32)
    
    x = Flatten()(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=x, outputs=output)

    model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"])
    return model

In [47]:
model = build_model()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 101)]             0         
                                                                 
 dense_6 (Dense)             (None, 4)                 408       
                                                                 
Total params: 408
Trainable params: 408
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(normed_train_data, train_l, batch_size = 16, epochs = 10, verbose=1, validation_data=(normed_val_data, val_l))

Epoch 1/10
1284/1284 [==============================] - 1s 1ms/step - loss: 0.2375 - accuracy: 0.8306 - val_loss: 0.2280 - val_accuracy: 0.8288
Epoch 2/10
1284/1284 [==============================] - 1s 1ms/step - loss: 0.2366 - accuracy: 0.8314 - val_loss: 0.2298 - val_accuracy: 0.8191
Epoch 3/10
1284/1284 [==============================] - 1s 1ms/step - loss: 0.2361 - accuracy: 0.8324 - val_loss: 0.2289 - val_accuracy: 0.8230
Epoch 4/10
1284/1284 [==============================] - 1s 1ms/step - loss: 0.2356 - accuracy: 0.8329 - val_loss: 0.2293 - val_accuracy: 0.8288
Epoch 5/10
1284/1284 [==============================] - 1s 975us/step - loss: 0.2352 - accuracy: 0.8323 - val_loss: 0.2273 - val_accuracy: 0.8327
Epoch 6/10
1284/1284 [==============================] - 1s 1ms/step - loss: 0.2349 - accuracy: 0.8338 - val_loss: 0.2274 - val_accuracy: 0.8210
Epoch 7/10
1284/1284 [==============================] - 1s 967us/step - loss: 0.2345 - accuracy: 0.8337 - val_loss: 0.2267 - val_accur

In [29]:
y_pred = model.predict(normed_test_data).argmax(axis=1)
encoder.fit(y_pred.reshape(-1, 1))
y_pred_one_hot = encoder.transform(y_pred.reshape(-1, 1)).toarray()

145/145 [==============================] - 0s 875us/step


array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [39]:
from sklearn.metrics import classification_report, accuracy_score
print("=== Classification Report ===")
print(classification_report(test_l, y_pred_one_hot))
print('\n')
print("=== Accuracy Score ===")
print(accuracy_score(test_l, y_pred_one_hot))

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.72      0.75       355
           1       0.88      0.90      0.89      1669
           2       0.81      0.84      0.83      1303
           3       0.82      0.78      0.80      1295

   micro avg       0.84      0.84      0.84      4622
   macro avg       0.82      0.81      0.82      4622
weighted avg       0.84      0.84      0.84      4622
 samples avg       0.84      0.84      0.84      4622



=== Accuracy Score ===
0.8357853742968412


In [37]:
y_test = encoder.inverse_transform(test_l).flatten()
y_pred = encoder.inverse_transform(y_pred_one_hot).flatten()

In [38]:
plot_cm(y_test, y_pred, normalize="true", labels= ['0', '1', '2', '3'], save=True,
directory=r'C:\Users\darias\Desktop\\', filename='LogisticRegression.png', title='Logistic Regression')